In [33]:
datapath1 = '/Users/dsea/Downloads/ubs-lauzhack-2024/account_booking_test.csv'
datapath2 = '/Users/dsea/Downloads/ubs-lauzhack-2024/account_booking_train.csv'
datapath3 = '/Users/dsea/Downloads/ubs-lauzhack-2024/external_parties_test.csv'
datapath4 = '/Users/dsea/Downloads/ubs-lauzhack-2024/external_parties_train.csv'

import pandas as pd
import numpy as np
from io import StringIO
from sentence_transformers import SentenceTransformer
# df_train = pd.read_csv('/dataset-ubs/external_parties_train.csv')
# infos_train = df_train['party_info_unstructured']
# infos_train.head()


#bucket_name = 'ubs-challenge'
#object_key_train = 'dataset-ubs/external_parties_train.csv'
#object_key_test = 'dataset-ubs/external_parties_test.csv'

# Read the CSV file from S3
#obj = s3.get_object(Bucket=bucket_name, Key=object_key_train)
df_train = pd.read_csv(datapath4)
infos_train = df_train['party_info_unstructured']
ids_train = df_train['transaction_reference_id']

#obj = s3.get_object(Bucket=bucket_name, Key=object_key_test)
df_test = pd.read_csv(datapath3)
infos_test = df_test['party_info_unstructured']
ids_test = df_test['transaction_reference_id']

print(ids_test)
print("-------")
print(ids_train)


from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

0          8f71ed0a819236b141978defe9a98700
1          dc9be336b81b971c04ca98ccdf99d51e
2          29c8ed895b8a220121168158a7447773
3          38ee5d04ad50ddf5f6abc67017201548
4          eef4f720e71bda0a6f1e7caa21a2c98c
                         ...               
1481667    a3b0502a9917c0da0c0343055d434c8f
1481668    38aac541d4b2cef7c05b584abfcca494
1481669    28f3dfc3160bae3b92c7c42e1082cf43
1481670    39f8569b0ced6d11fad95d628dd05d6a
1481671    ca917bf58677e7cd34b21d38a19ee17c
Name: transaction_reference_id, Length: 1481672, dtype: object
-------
0        04ff0d1c680189e3a80c92d86407f0f5
1        439ab0ad7380e6135ab2ff3fddd4a727
2        00cac12d41191a84f9e31aa731a83512
3        e4fba5f878dd3453e35973605a783a16
4        d03d7e4c31878b0255d39e8c3f0ab625
                       ...               
11059    7183d9c3700148c9527869948b685085
11060    b47b9ed0a8cc9fcc4c21ac368fe79757
11061    d61ff2b0184f32ad0021a313c6112b2e
11062    bec335b1b1bad8c55b7dce549cfd8de0
11063    c2a8edf088268bdd

In [34]:
import pickle
filePath = '/Users/dsea/Downloads/encoded_infos_test.pkl'

with open(filePath, 'rb') as f:
    encoded_infos_test = pickle.load(f)

print("Loaded encoded test data:")
print(encoded_infos_test.shape)

Loaded encoded test data:
(1481672, 384)


In [35]:
#how many nan values are there in the encoding?
print(np.isnan(encoded_infos_test).sum())

51


In [36]:
for i in range(len(encoded_infos_test)):
    if np.any(np.isnan(encoded_infos_test[i])):
        encoded_infos_test[i] = np.random.rand(len(encoded_infos_test[0]))

In [37]:
inizio_3 = 900000


encoded_infos_test[inizio_3:]

array([[-0.04334755,  0.02476263,  0.06969192, ..., -0.03989779,
        -0.04997503, -0.07210775],
       [-0.03549121,  0.00556043, -0.02851709, ..., -0.08507214,
        -0.05266014, -0.02874066],
       [-0.0055823 ,  0.03446376, -0.04563563, ...,  0.03965559,
        -0.0183202 , -0.00284681],
       ...,
       [-0.06026949, -0.0496878 , -0.01401023, ..., -0.02886893,
        -0.0917545 , -0.08177209],
       [-0.05589983,  0.08338015, -0.07226295, ...,  0.02313038,
        -0.03612743, -0.02633359],
       [ 0.02051861,  0.05351719, -0.06824867, ..., -0.04888948,
        -0.02920288, -0.02952541]], dtype=float32)

In [38]:
from sklearn.datasets import make_blobs
from sklearn.cluster import DBSCAN

eps = 0.63
clusterer = DBSCAN(eps, min_samples=2, metric='euclidean')
cluster_labels = clusterer.fit_predict(encoded_infos_test[inizio_3:].tolist())

# Example data generation (replace this with your actual encoded_infos_test data)
# Uncomment and replace with your actual data
# encoded_infos_test = ...

# Step 1: Apply UMAP for dimensionality reduction
#reducer = umap.UMAP(n_neighbors=30, min_dist=0, n_components=10, metric='euclidean', random_state=42)
#reduced_data = reducer.fit_transform(encoded_infos_train)
#precision, recall, f1 = compute_clustering_F1(df_train['external_id'][:10000], substitute_negative_ones(cluster_labels), num_pairs=2000000)
#print(precision, recall, f1, eps)

In [39]:
list_1 = list(-1 for i in range(900000))
len(list_1)
list_2 = list_1 + cluster_labels.tolist()
len(list_2)

1481672

In [41]:
import pickle

# Save the encoded data
with open('list_final.pkl', 'wb') as f:
    pickle.dump(list_2, f)

print("Encoded test data saved to 'encoded_infos_test_2.pkl'")

Encoded test data saved to 'encoded_infos_test_2.pkl'


In [33]:
import random
def compute_clustering_F1(A, B, num_pairs=10000):
    """
    Computes the precision, recall, and F1-score for a clustering algorithm.

    This function evaluates the performance of a clustering algorithm by comparing 
    its predicted labels (`B`) to the ground truth labels (`A`). It uses a 
    pairwise approach, sampling a specified number of data point pairs and 
    categorizing them based on whether the algorithm correctly identifies 
    whether the points belong to the same cluster.

    Args:
        A (list): A list of ground truth labels for the data points.
        B (list): A list of predicted labels from the clustering algorithm.
        num_pairs (int, optional): The number of random data point pairs to 
            sample for evaluation. Defaults to 10000.

    Returns:
        tuple: A tuple containing the precision, recall, and F1-score, in that order.

    Raises:
        ValueError: If `A` and `B` have different lengths.

    Examples:
        >>> A = [1, 2, 3, 4, 5, 5, 6, 7, 4, 4, 3, 2]
        >>> B = [10, 20, 30, 40, 50, 50, 60, 70, 40, 40, 30, 20]
        >>> precision, recall, f1 = compute_clustering_F1(A, B)
        >>> print(f"Precision: {precision:.4f}")
        >>> print(f"Recall: {recall:.4f}")
        >>> print(f"F1-score: {f1:.4f}")
    """
    n = len(A)
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0
    
    for _ in range(num_pairs):
        i, j = random.sample(range(n), 2)
        
        same_label_a = A[i] == A[j]
        same_label_b = B[i] == B[j]
        
        if same_label_a and same_label_b:
            true_positives += 1
        elif not same_label_a and same_label_b:
            false_positives += 1
        elif same_label_a and not same_label_b:
            false_negatives += 1
        else:
            true_negatives += 1
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1


def substitute_negative_ones(input_list):
    """
    Substitutes -1 values in a list with distinct new numbers.

    Args:
    input_list: A list of integers.

    Returns:
    A new list with -1 values replaced by distinct new numbers.
    """
    new_list = []

    count = -2
    for num in input_list:
        if num == -1:
            new_list.append(count)
            count -= 1
        else:
            new_list.append(num)
    return new_list


In [46]:
    precision, recall, f1 = compute_clustering_F1(df_train['external_id'][:5000], substitute_negative_ones(cluster_labels), num_pairs=1000000)
    print(precision, recall, f1, eps)


0.9904761904761905 0.5279187817258884 0.6887417218543047
